![alt text](http://quantstack.net/assets/images/xframe.svg)

<center> <h1>xframe is a dataframe for C++, based on xtensor and xtl</h1> </center>

In [ ]:
#include <string>
#include <iostream>

#include "xtl/xbasic_fixed_string.hpp"

#include "xtensor/xio.hpp"

#include "xframe/xvariable.hpp"

In [ ]:
using fstring = xtl::xfixed_string<55>;

using data_type = xt::xoptional_assembly<xt::xarray<double>, xt::xarray<bool>>;
using coordinate_type = xf::xcoordinate<fstring, data_type::size_type>;

using variable_type = xf::xvariable<coordinate_type, data_type>;
using dimension_type = xf::xdimension<fstring, std::size_t>;
using saxis_type = xf::xaxis<fstring, std::size_t>; 
using iaxis_type = xf::xaxis<int, std::size_t>;

In [ ]:
fstring abscissa = "abscissa";
fstring ordinate = "ordinate";

In [ ]:
data_type data = {{ 1., 2., 3.},
                  { 4., 5., 6.},
                  { 7., 8., 9.}};
data(0, 2).has_value() = false;
data(1, 0).has_value() = false;

In [ ]:
variable_type variable = variable_type(
    data, 
    {
        {abscissa, saxis_type({"a", "c", "d"})},
        {ordinate, iaxis_type({1, 2, 4})}
    }, 
    dimension_type({abscissa, ordinate})
);

In [ ]:
std::cout << variable.select({{"abscissa", "a"}, {"ordinate", 1}});

In [ ]:
std::cout << variable.select({{"abscissa", "c"}, {"ordinate", 2}});

In [ ]:
std::cout << variable.locate("c", 2);

In [ ]:
std::cout << variable.data();